In [1]:
import pandas as pd
data = pd.read_csv('C:/Users/phsivale/Documents/Trainings/flaskPOC/bank.csv')

data.head()

,Age,Experience,Income,Home,PersonalLoan
0,25,1,49,a,0
1,45,19,34,a,0
2,39,15,11,b,0
3,35,9,100,c,0
4,35,8,45,c,0


In [2]:
data.shape

(5000, 5)

In [3]:
pd.get_dummies(data).head()

,Age,Experience,Income,PersonalLoan,Home_a,Home_b,Home_c
0,25,1,49,0,1,0,0
1,45,19,34,0,1,0,0
2,39,15,11,0,0,1,0
3,35,9,100,0,0,0,1
4,35,8,45,0,0,0,1


In [6]:
pd.get_dummies(data.loc[:2,]).head()

,Age,Experience,Income,PersonalLoan,Home_a,Home_b
0,25,1,49,0,1,0
1,45,19,34,0,1,0
2,39,15,11,0,0,1


#### Either use Label Enc with One Hot Encoder or just use Label Binarizer to convert categorical features to numeric features

## Option 1. Label Enc and One Hot Enc

In [1]:
from sklearn.preprocessing import LabelEncoder

In [3]:
enc = LabelEncoder()

In [4]:
enc.fit(data['Home'])

LabelEncoder()

In [5]:
data['Home_Cat'] = enc.transform(data['Home'])

In [6]:
data.head()

,Age,Experience,Income,Home,PersonalLoan,Home_Cat
0,25,1,49,a,0,0
1,45,19,34,a,0,0
2,39,15,11,b,0,1
3,35,9,100,c,0,2
4,35,8,45,c,0,2


In [7]:
data.drop(['Home'],axis=1,inplace=True)

In [8]:
data['Home_Cat'] = data['Home_Cat'].astype('category')

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
enc2 = OneHotEncoder(sparse=False)

In [11]:
enc2.fit(pd.DataFrame(data['Home_Cat']))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [12]:
data2 = enc2.transform(pd.DataFrame(data['Home_Cat']))

In [13]:
pd.DataFrame(data2).head()


,0,1,2
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0


## Option 2 : Label Binarizer (Easier One)

In [17]:
from sklearn.preprocessing import LabelBinarizer

In [18]:
binz = LabelBinarizer()

In [9]:
data.head()

,Age,Experience,Income,Home,PersonalLoan
0,25,1,49,a,0
1,45,19,34,a,0
2,39,15,11,b,0
3,35,9,100,c,0
4,35,8,45,c,0


In [19]:
binz.fit(data['Home'])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [20]:
data_bin = binz.transform(data['Home'])

In [21]:
data_bin = pd.DataFrame(data_bin)

In [22]:
data_bin.head()

,0,1,2
0,1,0,0
1,1,0,0
2,0,1,0
3,0,0,1
4,0,0,1


###### New levels in categorical variables are handled gracefully. The schema is retained 

In [14]:
test = pd.DataFrame({"Age":[20,10],"Experience":[10,5],"Income":10,"Home":["p",'a']})

In [15]:
test

,Age,Experience,Income,Home
0,20,10,10,p
1,10,5,10,a


In [6]:
pd.get_dummies(test)

,Age,Experience,Income,Home_a,Home_p
0,20,10,10,0,1
1,10,5,10,1,0


In [16]:
pd.DataFrame(binz.transform(test['Home']))

,0,1,2
0,0,0,0
1,1,0,0


In [12]:
data.head()

,Age,Experience,Income,Home,PersonalLoan
0,25,1,49,a,0
1,45,19,34,a,0
2,39,15,11,b,0
3,35,9,100,c,0
4,35,8,45,c,0


### Dropping 'Home' column

In [23]:
data.drop(['Home'],axis=1,inplace = True)

In [24]:
data_new = pd.concat([data,data_bin],axis=1)

In [25]:
data_new.head()

,Age,Experience,Income,PersonalLoan,0,1,2
0,25,1,49,0,1,0,0
1,45,19,34,0,1,0,0
2,39,15,11,0,0,1,0
3,35,9,100,0,0,0,1
4,35,8,45,0,0,0,1


### Building the RF Model

In [26]:
from sklearn.ensemble import RandomForestClassifier

C:\Users\phsivale\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [27]:
X = data_new.drop(['PersonalLoan'],axis=1)
y = data_new['PersonalLoan']

In [28]:
clf = RandomForestClassifier()
clf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Dumping Objects. To be used during predictions

In [29]:
import pickle

In [30]:
with open('C:/Users/phsivale/Documents/Trainings/flaskPOC/model24Mar.pkl','wb') as f:
    pickle.dump(clf,f)

In [31]:
with open('C:/Users/phsivale/Documents/Trainings/flaskPOC/bin24Mar.pkl','wb') as f:
    pickle.dump(binz,f)